In [3]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [5]:
from collections import defaultdict
from itertools import combinations
import json
import glob

In [6]:
root = "/Users/rahulmehta/Desktop/MultiSent/datasets/"

In [38]:
all_datasets = glob.glob(root + 'split_merged_dataset/*_train.jsonl')
all_datasets
all_datasets[3].split('/')[-1][:-6].split('_')[0]

'ta'

In [11]:
same_sent_dict = defaultdict(lambda: defaultdict(int))
same_entity_dict = defaultdict(lambda: defaultdict(int))
total_raw_facts = defaultdict(int)
total_unique_facts = defaultdict(int)
total_fact_pairs = defaultdict(int)

for dataset in all_datasets:
    lang = dataset.split('/')[-1][:-6].split('_')[0]
    with open(dataset) as f:
        for line in f:
            data = json.loads(line)
            for sent_facts in data['facts_list']:
                facts = [i[0] for i in sent_facts]
                fact_pairs = list(combinations(set(facts), 2))
                for pair in fact_pairs:
                    k = "|".join(sorted(pair))
                    same_sent_dict[lang][k]+=1
            #total_raw_facts[lang]+=len(facts)
            #total_unique_facts[lang]+=len(set(facts))
            all_facts = [i[0] for sent_facts in data['facts_list'] for i in sent_facts]
            all_fact_pairs = list(combinations(all_facts, 2))
            for pair in all_fact_pairs:
                k = "|".join(sorted(pair))
                same_entity_dict[lang][k]+=1

In [12]:
same_sent_dict['as']

defaultdict(int,
            {'educated at|occupation': 37,
             'occupation|position held': 94,
             'occupation|sport': 94,
             'date of death|name in native language': 14,
             'conflict|occupation': 3,
             'conflict|country of citizenship': 1,
             'conflict|founded by': 2,
             'country of citizenship|occupation': 7,
             'founded by|occupation': 4,
             'country of citizenship|founded by': 1,
             'conflict|participant': 6,
             'conflict|member of political party': 1,
             'member of political party|participant': 1,
             'conflict|date of birth': 1,
             'date of birth|participant': 2,
             'date of birth|occupation': 189,
             'date of birth|founded by': 3,
             'occupation|participant': 3,
             'founded by|participant': 1,
             'instrument|occupation': 86,
             'author|occupation': 32,
             'languages spoken, 

In [53]:
same_entity_dict['as']

defaultdict(int,
            {'place of birth|position held': 33,
             'languages spoken, written or signed|position held': 8,
             'date of birth|position held': 65,
             'award received|position held': 24,
             'occupation|position held': 109,
             'author|position held': 2,
             'date of death|position held': 39,
             'languages spoken, written or signed|place of birth': 40,
             'date of birth|place of birth': 192,
             'award received|place of birth': 74,
             'occupation|place of birth': 248,
             'author|place of birth': 27,
             'date of death|place of birth': 67,
             'date of birth|languages spoken, written or signed': 58,
             'award received|languages spoken, written or signed': 38,
             'languages spoken, written or signed|occupation': 113,
             'author|languages spoken, written or signed': 28,
             'date of death|languages spoken, written

In [13]:
prob_dict = defaultdict(lambda: defaultdict(int))

In [19]:
#prob_dict

In [23]:
for lang in same_sent_dict:
    for pair in same_sent_dict[lang]:
        prob_dict[lang][pair] = same_sent_dict[lang][pair]/same_entity_dict[lang][pair]

In [42]:
prob_dict

defaultdict(<function __main__.<lambda>()>,
            {'pa': defaultdict(int,
                         {'family|father': 0.32142857142857145,
                          'family|religion': 0.5,
                          'family|occupation': 0.15873015873015872,
                          'father|religion': 0.2,
                          'father|occupation': 0.08021390374331551,
                          'occupation|religion': 0.14046822742474915,
                          'child|religion': 0.14285714285714285,
                          'award received|occupation': 0.08226415094339623,
                          'award received|date of death': 0.10618066561014262,
                          'date of death|occupation': 0.2638049593665347,
                          'languages spoken, written or signed|occupation': 0.17113364722926475,
                          'date of birth|languages spoken, written or signed': 0.3060556464811784,
                          'date of death|languages spoken, w

In [40]:
for lang in prob_dict:
    with open(root + f'fact_pair_probs/{lang}.tsv', 'w') as f:
        for pair in prob_dict[lang]:
            f.write(f'{pair}\t{prob_dict[lang][pair]}\n')

In [41]:
langs = ['en', 'hi']
for lang in langs:
    print(lang)
    top = [i for i in sorted(list(prob_dict[lang].items()), key=lambda x: x[1], reverse=True) if i[1]!=1]
    print('TOP 5')
    for i in top[:5]:
        print(i)
    
    print('BOTTOM 5')
    for i in top[-1:-6:-1]:
        print(i)
    


en
TOP 5
('doctoral advisor|student of', 0.9655172413793104)
('instrument|voice actor', 0.8888888888888888)
('chief executive officer|director / manager', 0.875)
('date of birth|short name', 0.8461538461538461)
('family|time period', 0.8333333333333334)
BOTTOM 5
('nominated for|place of birth', 0.0018484288354898336)
('cause of death|educated at', 0.002380952380952381)
('employer|nominated for', 0.0024330900243309003)
('filmography|nominated for', 0.0028011204481792717)
('educated at|place of burial', 0.003257328990228013)
hi
TOP 5
('ethnic group|religion', 0.8571428571428571)
('birth name|related category', 0.8333333333333334)
('employer|professorship', 0.8)
('country for sport|country of citizenship', 0.8)
('head of state|participant', 0.75)
BOTTOM 5
('cast member|educated at', 0.003676470588235294)
('named after|nominated for', 0.0038910505836575876)
('date of birth|nominated for', 0.0055248618784530384)
('cast member|father', 0.007352941176470588)
('cast member|place of birth', 0.0